<a id="top"></a>
### Sumário:

---

<b>* [RG e Faturamento](#rg)</b> <br>
<b>* [Beneficiários/Colaboradores](#ben)</b> <br>
<b>* [Eventos](#ev)</b> <br>
<b>* [Mensalidades](#men)</b> <br>
<b>* [Coparticipação](#cop)</b> <br>

In [1]:
import pandas as pd
import unidecode
%load_ext autoreload
%autoreload 2

In [2]:
from lib.parse_file import *

In [3]:
%cd ../../
db = WellbeDatabase("db_conf.yaml")
%cd -

D:\wellbe_scripts
WellbeDatabase: banco stillgood no endereço dev-wellbe.mysql.database.azure.com, dev Otavio
Verificando conexão: Ok
Instância WellbeDatabase criada com sucesso!
D:\wellbe_scripts\scripts\operadora3


Atualizações em massa: deixar empresa_id = None

In [ ]:
empresa_id = 123
operadora = 'Bradesco'

<a id="rg"></a>

---

<h3>RG e Faturamento</h3>

---

<b>* [Sumário](#top)</b>

In [ ]:
rg_paths = [

    r'H:\.shortcut-targets-by-id\1DS9OnHUP3iNzEtSIUfF_MeW6WOANkzsI\Bases\Unnimax\Doremus\Unimed Guarulhos\RG\2023\11\*'

]
rg_paths = [item.replace('\\','/') for sublist in [glob(i) for i in rg_paths] for item in sublist if ".ini" not in item]
rg_paths[:]

In [ ]:
%%time
debug = False if len(rg_paths) > 10 else True
rgs = []
for file_path in tqdm(rg_paths):
    try:
        rg = parse_file(file_path, empresa_id, operadora, 'rg', db=db, debug=debug)
        success(f'Ok {file_path}\n')
        rgs.append(rg)
    except pd.errors.EmptyDataError:
        warning(f'Vazio {file_path}\n')
    except Exception as e:
        error(f'Erro {file_path}, {e}\n')
        #raise e
rgs = pd.concat(rgs, ignore_index=True)

In [ ]:
rgs.sort_values(by=["empresa_id", "data"], inplace=True, ignore_index=True)
rgs

In [ ]:
# rgs = rgs[rgs["data"] == pd.to_datetime('2023-07-01')]
# rgs.data.unique()

In [ ]:
rgs

---
Opção para selecionar última competência de acordo com o banco

In [ ]:
final = []
for empresa in rgs.empresa_id.unique():
    temp = rgs[rgs['empresa_id'] == empresa]
    temp = temp[temp["data"] > db.runQueryWithPandas(f"SELECT MAX(data) AS 'maximo' FROM BI_Validador WHERE empresa_id = {empresa} AND operadora LIKE '%{rgs[rgs['empresa_id'] == empresa]['operadora'].unique()[0]}%'").maximo[0]]
    final.append(temp)

rgs = pd.concat(final, ignore_index=True)
rgs

---
### Cards

In [ ]:
get_cards(rgs, db).print_cards(db)

---

In [ ]:
rgs_backup = rgs.copy(deep=True)
#rgs = rgs_backup.copy(deep=True)

In [ ]:
print(len(rgs))
if "subestipulante" in rgs.columns:
    rgs.drop_duplicates(inplace=True, subset=['data', 'faturamento', 'sinistro', 'vidas', 'operadora', 'subestipulante', 'empresa_id'])
else:
    rgs.drop_duplicates(inplace=True, subset=['data', 'faturamento', 'sinistro', 'vidas', 'operadora', 'empresa_id'])
print(len(rgs))

In [ ]:
# rgs = rgs[rgs["data"] >= pd.to_datetime('2023-05-01')]
# rgs

In [ ]:
#rgs[rgs['empresa_id'].isna()]['fcode'].unique()
#rgs["data"].unique()
#rgs['subestipulante'].unique()
rgs["empresa_id"].unique()
#rgs["operadora"].unique()

In [ ]:
rgs

In [ ]:
funcao = get_function(operadora, "rg")

In [ ]:
rgs = run_function(funcao, df=rgs, db=db, operadora=operadora, empresa_id=empresa_id)

In [ ]:
if type(rgs) == list:
    display(rgs[0])
else:
    display(rgs.head())

In [ ]:
rgs

In [ ]:
%%time
if type(rgs) == list:
    rgs[0].to_sql(con=db.getEngine(), name="BI_Validador", if_exists="append", index=False)
else:
    rgs.to_sql(con=db.getEngine(), name="BI_Validador", if_exists="append", index=False)

Se estiver por faturamento, recomenda-se rodar as queries após subir o faturamento

In [ ]:
queries = get_queries(operadora, "rg")

In [ ]:
run_queries(queries, df=rgs, db=db, operadora=operadora, empresa_id=empresa_id, verbose=False)

---

#### Faturamento

---

In [ ]:
if type(rgs) == list:
    fat = rgs[1].copy(deep=True) # sulamerica
else:
    fat = rgs.copy(deep=True)

In [ ]:
fat.rename(columns={"faturamento": "valor"}, inplace=True)
fat.drop(columns=["sinistro", "sinistralidade", "vidas", "bkp", "bkp2", "bkp3"], axis=1, errors='ignore', inplace=True)
fat

In [ ]:
#fat = fat[~fat["empresa_id"].isin(["55", "1253", "1396"])]

In [ ]:
fat.to_sql(con=db.getEngine(), name="BI_Faturamento", if_exists="append", index=False)

In [ ]:
run_queries(queries, df=rgs, db=db, operadora=operadora, empresa_id=empresa_id, verbose=False)

Mensalidades: Para empresas que estão por mensalidades por causa de uma outra operadora

In [ ]:
if type(rgs) == list:
    mens = rgs[2].copy(deep=True) # sulamerica
    mens.rename(columns={"faturamento":"mensalidade", "data":"data_realizacao"}, inplace=True)
else:
    mens = rgs.copy(deep=True)
    mens.rename(columns={"valor":"mensalidade", "data":"data_realizacao"}, inplace=True)

In [ ]:
mens.drop(columns=["sinistro", "sinistralidade", "vidas"], axis=1, errors='ignore', inplace=True)
mens

In [ ]:
mens.empresa_id.unique()

In [ ]:
# mens = mens[mens["empresa_id"].isin(['49', '76', '2147', '1554'])]
# mens

In [ ]:
mens.to_sql(con=db.getEngine(), name="BI_Mensalidades", if_exists="append", index=False)

In [ ]:
run_queries(queries, df=rgs, db=db, operadora=operadora, empresa_id=empresa_id, verbose=False)

<a id="ben"></a>

---

<h3>Beneficiários e Colaboradores</h3>

---

<b>* [Sumário](#top)</b> <br>
<b>* [RG e Faturamento](#rg)</b> <br>
<b>* [Eventos](#ev)</b> <br>
<b>* [Mensalidades](#men)</b> <br>
<b>* [Coparticipação](#cop)</b> <br>

In [ ]:
ben_paths = [

    r'H:\.shortcut-targets-by-id\1DS9OnHUP3iNzEtSIUfF_MeW6WOANkzsI\Bases\Unnimax\Kion\bradesco\beneficiarios\2023\10\*',
    #r'H:\.shortcut-targets-by-id\1DS9OnHUP3iNzEtSIUfF_MeW6WOANkzsI\Bases\Unnimax\Kion\bradesco\beneficiarios\2023\11\*',

]
ben_paths = [item.replace('\\','/') for sublist in [glob(i) for i in ben_paths] for item in sublist if ".ini" not in item]
ben_paths

In [ ]:
if operadora.lower() == 'amil':
    ben_paths = drop_fcode_duplicated(ben_paths, encoding="latin1")

In [ ]:
%%time 
benes, cols = [], []
debug = False if len(ben_paths) > 10 else True
for file_path in tqdm(ben_paths):
    print("---\n\x1b[38;5;3m\u2794 Rodando para arquivo: {} \x1b[0m".format(file_path))
    try:
        col, ben = parse_file(file_path, empresa_id, operadora, 'ben', db=db, debug=debug)
        print('\n\x1b[38;5;42m\u2714 Ok \u2794 {}\x1b[0m\n'.format(file_path))
        cols.append(col)
        benes.append(ben)
    except pd.errors.EmptyDataError:
        warning(f'Vazio {file_path}\n')
    except Exception as e:
        error(f'\nErro {file_path}, {e}\n')
        if len(ben_paths) <= 10:
            raise e

cols = pd.concat(cols, ignore_index=True)
benes = pd.concat(benes, ignore_index=True)
result = pd.concat([cols, benes], ignore_index=True)

In [ ]:
result.empresa_id.unique()

In [ ]:
# cols
cols[['nome', 'inclusao_plano_real', 'cancelamento_plano_real']][cols['inclusao_plano_real'] > cols['cancelamento_plano_real']].sort_values(by='cancelamento_plano_real').head(50)

In [ ]:
# benes
benes[['nome', 'inclusao_plano_real', 'cancelamento_plano_real']][benes['inclusao_plano_real'] > benes['cancelamento_plano_real']].sort_values(by='cancelamento_plano_real').head(50)

---
### Verificar com RG, caso saia em branco reestabelecer conexão com o DB

Verificar dados com o validador (para verificar os dados recém atualizados é necessário refazer a conexão com o banco!)

Com suporte para múltiplas empresas, mas ainda sem para retroativo

In [ ]:
check_with_validador_ben(result, db)

---
### Cards/ Se já temos dados no banco

In [ ]:
get_cards(result, db).print_cards(db)

In [ ]:
cheeck_with_db(result).is_in_db(tabela = 'result', db = db)

---

In [ ]:
checkpoint_ben(cols, benes)

In [ ]:
result.nome_plano.unique()

In [ ]:
result

In [ ]:
if "competencia" in result.columns:
    display(result[["empresa_id", "competencia", "cpf"]].groupby(by=["empresa_id", "competencia"]).nunique().reset_index().sort_values('competencia'))
else:
    display(result[["empresa_id", "inclusao_plano", "cpf"]].groupby(by=["empresa_id", "inclusao_plano"]).nunique().reset_index().sort_values('competencia'))

In [ ]:
#result['competencia'].unique()
#result['inclusao_plano'].unique()
# result['inclusao_plano_real'].unique()
#result['cancelamento_plano'].unique()
#result['status'].unique()
result['cod_plano'].unique()
#result['titularidade'].unique()
#result['fcode'].unique()
#result['operadora'].unique()
#result['empresa_id'].unique()

In [ ]:
%%time
cols = set_carimbo(cols, db=db, script_name='operadora3/beneficiarios')
benes = set_carimbo(benes, db=db, script_name='operadora3/beneficiarios')

In [ ]:
funcao = get_function(operadora, "ben")

In [ ]:
benes, cols = run_function(funcao, df=[benes, cols], db=db, operadora=operadora, empresa_id=empresa_id)

In [ ]:
# cols[['nome', 'inclusao_plano_real', 'cancelamento_plano_real']].sort_values(by='cancelamento_plano_real').head(50)

In [ ]:
%%time 
cols.to_sql(
    con=db.getEngine(), name="BI_Colaborador", if_exists="append", index=False
)
benes.to_sql(
    con=db.getEngine(), name="BI_Beneficiario", if_exists="append", index=False
)

In [ ]:
queries = get_queries(operadora, "ben")

In [ ]:
%%time 
run_queries(queries, df=[benes, cols], db=db, operadora=operadora, empresa_id=empresa_id, verbose=False)

<a id="ev"></a>

---

<h3>Eventos</h3>

---

<b>* [Sumário](#top)</b> <br>
<b>* [RG e Faturamento](#rg)</b> <br>
<b>* [Beneficiarios e Colaboradores](#ben)</b> <br>
<b>* [Mensalidades](#men)</b> <br>
<b>* [Coparticipação](#cop)</b> <br>

In [ ]:
ev_paths = [

    ''
]
ev_paths = [item.replace('\\','/') for sublist in [glob(i) for i in ev_paths] for item in sublist if ".ini" not in item]
ev_paths[:]

In [ ]:
# rodar p/ Amil
if operadora.lower() == 'amil':
    ev_paths = drop_fcode_duplicated(ev_paths, encoding="latin1")

In [ ]:
%%time
evs = []
debug = False if len(ev_paths) > 10 else True
for file_path in tqdm(ev_paths):
    print("---\n\x1b[38;5;3m\u2794 Rodando para arquivo: {} \x1b[0m".format(file_path))
    try:
        ev = parse_file(file_path, empresa_id, operadora, 'ev', db=db, debug=debug)
        print('\n\x1b[38;5;42m\u2714 Ok \u2794 {}\x1b[0m\n'.format(file_path))
        evs.append(ev)
    except pd.errors.EmptyDataError:
        warning(f'Vazio {file_path}\n')
    #except Exception as e:
    #    error(f'Erro {file_path}, {e}\n')
    #    if len(ev_paths) <= 10:
    #        raise e
evs = pd.concat(evs, ignore_index=True)

In [ ]:
if operadora.lower() == 'amil':
    display(evs[evs["cpf_titular"].isna()]['descricao'].unique()) # Amil, p/ verificar PEONA
elif operadora.lower() == 'sulamerica':
    display(evs[pd.isnull(evs['nome_titular'])]) # Sulamerica, verificar não transpostos
elif operadora.lower() == 'central nacional unimed':
    display(evs.loc[(evs['titularidade'].isna())]) # CNU, verificar não transpostos

---
Verificar dados com o validador (para verificar os dados recém atualizados é necessário refazer a conexão com o banco!)

In [ ]:
evs

In [ ]:
check_with_validador_evs(evs, db)

In [ ]:
# evs = evs[evs['competencia'] == pd.to_datetime('2023-08-01')]

In [ ]:
# evs = evs[~evs["empresa_id"].isin(["2139"])]

In [ ]:
# display(evs[["competencia", "empresa_id", "valor"]].groupby(["empresa_id","competencia"]).sum())

---
### Cards / Se já temos no banco

In [ ]:
get_cards(evs, db).print_cards(db)

In [ ]:
cheeck_with_db(evs).is_in_db(tabela = 'evs', db = db)

---

In [ ]:
# categorias
display(evs[["categoria", "cpf"]].groupby(by=["categoria"], dropna=False).nunique().sort_values('cpf', ascending=False))

In [ ]:
#evs['titularidade'].unique()
#evs['sexo'].unique()
evs['subestipulante'].unique()
#evs['status'].unique()
#evs['nascimento'].unique()
# evs['categoria'].unique()
#evs['quantidade'].unique()
#evs['subcategoria'].unique()
#evs['data_pagamento'].unique()
#evs[evs['cpf_real'].isna()]
#evs['alta_internacao'].unique()
#evs['data_inicio'].unique()
#evs['competencia'].unique()
#evs['operadora'].unique()
#evs['empresa_id'].unique()
#evs['cpf_real'].unique()

In [ ]:
evs = set_carimbo(evs, db=db, script_name='operadora3/eventos')

In [ ]:
funcao = get_function(operadora, "ev")

In [ ]:
evs = run_function(funcao, df=evs, db=db, operadora=operadora, empresa_id=empresa_id)

In [ ]:
%%time
evs.to_sql(con=db.getEngine(), name='BI_Eventos', if_exists='append', index=False)

In [ ]:
queries = get_queries(operadora, "ev")

In [ ]:
%%time
run_queries(queries, df=evs, db=db, operadora=operadora, empresa_id=empresa_id, verbose=False)

<a id="men"></a>

---

<h3>Mensalidades</h3>

---

<b>* [Sumário](#top)</b> <br>
<b>* [RG e Faturamento](#rg)</b> <br>
<b>* [Beneficiarios e Colaboradores](#ben)</b> <br>
<b>* [Eventos](#ev)</b> <br>
<b>* [Coparticipação](#cop)</b> <br>

In [ ]:
mens_paths = [
	'',
]
mens_paths = [item.replace('\\','/') for sublist in [glob(i) for i in mens_paths] for item in sublist if ".ini" not in item]
mens_paths

In [ ]:
# rodar p/ Amil
if operadora.lower() == 'amil':
    mens_paths = drop_fcode_duplicated(mens_paths, encoding="cp1254")

In [ ]:
%%time
mens = []
debug = False if len(mens_paths) > 10 else True
for file_path in tqdm(mens_paths):
    print("---\n\x1b[38;5;3m\u2794 Rodando para arquivo: {} \x1b[0m".format(file_path))
    try:
        men = parse_file(file_path, empresa_id, operadora, 'men', db=db, debug=debug)
        print('\n\x1b[38;5;42m\u2714 Ok \u2794 {}\x1b[0m\n'.format(file_path))
        mens.append(men)
    except pd.errors.EmptyDataError:
        warning(f'Vazio {file_path}\n')
    #except Exception as e:
    #    error(f'Erro {file_path}, {e}\n')
    #    if len(mens_paths) <= 10:
    #        raise e
mens = pd.concat(mens, ignore_index=True)

---
### Cards/ Se já temos dados no banco

In [ ]:
get_cards(mens, db).print_cards(db)

In [ ]:
cheeck_with_db(mens).is_in_db(tabela = 'mensalidade', db = db)

---

In [ ]:
checkpoint_men(mens)

In [ ]:
mens.operadora.unique()

In [ ]:
#mens['plano'].unique()
#mens['empresa_id'].unique()
#mens['titularidade'].unique()
#mens['cpf_real_titular'].unique()

In [ ]:
if "outros" in mens.columns and "coparticipacao" in mens.columns:
    display(mens[["data_realizacao", "empresa_id", "mensalidade", "outros", "coparticipacao"]].groupby(["empresa_id","data_realizacao"]).sum())
elif "coparticipacao" in mens.columns:
    display(mens[["data_realizacao", "empresa_id", "mensalidade", "coparticipacao"]].groupby(["empresa_id","data_realizacao"]).sum())
elif "outros" in mens.columns:
    display(mens[["data_realizacao", "empresa_id", "mensalidade", "outros"]].groupby(["empresa_id","data_realizacao"]).sum())
else:
    display(mens[["data_realizacao", "empresa_id", "mensalidade"]].groupby(["empresa_id","data_realizacao"]).sum())

In [ ]:
mens = set_carimbo(mens, db=db, script_name='operadora3/mensalidades')

In [ ]:
funcao = get_function(operadora, "men")

In [ ]:
mens = run_function(funcao, df=mens, db=db, operadora=operadora, empresa_id=empresa_id)

In [ ]:
#mens = fix_mensalidades(db=db, df=mens, force=False)

In [ ]:
mens

In [ ]:
mens.to_sql(con=db.getEngine(), name="BI_Mensalidades", if_exists="append", index=False)

In [ ]:
queries = get_queries(operadora, "men")

In [ ]:
run_queries(queries, df=mens, db=db, operadora=operadora, empresa_id=empresa_id, verbose=False)

---
# Copay
---

<b>* [Sumário](#top)</b> <br>
<b>* [RG e Faturamento](#rg)</b> <br>
<b>* [Beneficiarios e Colaboradores](#ben)</b> <br>
<b>* [Eventos](#ev)</b> <br>
<b>* [Mensalidades](#men)</b> <br>

In [ ]:
cop_paths = [
	''
]
cop_paths = [item.replace('\\','/') for sublist in [glob(i) for i in cop_paths] for item in sublist if ".ini" not in item]
cop_paths

In [ ]:
%%time
cops = []
debug = False if len(cop_paths) > 10 else True
for file_path in tqdm(cop_paths):
    try:
        cop = parse_file(file_path, empresa_id, operadora, 'cop', db=db, debug=True)
        success(f'Ok {file_path}\n')
        cops.append(cop)
    except pd.errors.EmptyDataError:
        warning(f'Vazio {file_path}\n')
    except Exception as e:
        error(f'Erro {file_path}, {e}\n')
        if len(cop_paths) <= 10:
            raise e
cops = pd.concat(cops, ignore_index=True)

In [ ]:
cops

---
### Cards/ Se já temos dados no banco

In [ ]:
get_cards(cops, db).print_cards(db)

In [ ]:
cheeck_with_db(cops).is_in_db(tabela = 'copay', db = db)

---

In [ ]:
checkpoint_cop(cops)

In [ ]:
if "outros" in cops.columns and "mensalidade" in cops.columns:
    display(cops[["data_realizacao", "empresa_id", "mensalidade", "outros", "coparticipacao"]].groupby(["empresa_id","data_realizacao"]).sum())
elif "mensalidade" in cops.columns:
    display(cops[["data_realizacao", "empresa_id", "mensalidade", "coparticipacao"]].groupby(["empresa_id","data_realizacao"]).sum().reset_index().sort_values(by="empresa_id", ascending = False))
elif "outros" in cops.columns:
    display(cops[["data_realizacao", "empresa_id", "coparticipacao", "outros"]].groupby(["empresa_id","data_realizacao"]).sum().reset_index())
else:
    display(cops[["data_realizacao", "empresa_id", "coparticipacao"]].groupby(["empresa_id","data_realizacao"]).sum())

In [ ]:
#cops = cops[cops['data_realizacao'] == '2021-03-01']

In [ ]:
cops = set_carimbo(cops, db=db, script_name='operadora3/coparticipacao')

In [ ]:
funcao = get_function(operadora, "cop")

In [ ]:
cops = run_function(funcao, df=cops, db=db, operadora=operadora, empresa_id=empresa_id)

In [ ]:
cops.to_sql(con=db.getEngine(), name='BI_Mensalidades', if_exists='append', index=False)

In [ ]:
queries = get_queries(operadora, "cop")

In [ ]:
run_queries(queries, df=cops, db=db, operadora=operadora, empresa_id=empresa_id, verbose=False)

<b>* [Sumário](#top)</b> <br>
<b>* [RG e Faturamento](#rg)</b> <br>
<b>* [Beneficiarios e Colaboradores](#ben)</b> <br>
<b>* [Eventos](#ev)</b> <br>
<b>* [Mensalidades](#men)</b> <br>
<b>* [Coparticipação](#cop)</b> <br>